In [4]:
import numpy as np
import pandas as pd
import dill as dl
import sys
import pickle
import itertools
import re
import os
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as sm_f
#from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
#from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
import plotnine

In [5]:
pd.set_option("max_columns", None)
pd.set_option("max_rows", None)

In [6]:
def series_in(series, within):
    return pd.Series(data=[True if i in within else False for i in series.values], index=series.index)

In [7]:
mapping_list = ["Приморский край", "Амурская область", "Кировская область", "Липецкая область", "Мурманская область", "Новгородская область", "Костромская область", "Республика Алтай", "Республика Марий Эл", "Хабаровский край", "Тульская область"]

In [8]:
#mapping_list = ["Приморский край", "Мурманская область", "Новгородская область", "Костромская область", "Республика Алтай", "Республика Марий Эл", "Хабаровский край", "Тульская область"]

In [9]:
maj_data = pd.read_csv("D:/DZ/Course_5/Курсовая/data/maj_data.csv", index_col=0)

# First time incumbents

In [10]:
def sort_out_func(mapping, data):
    data_to_collect = pd.DataFrame()
    for i in mapping:
        data_to_collect = data_to_collect.append(data.loc[data.region == i])
    #data_to_collect.to_csv("D:/DZ/Course_5/Курсовая/data/treatment_group_data.csv")
    return data_to_collect

In [11]:
data = sort_out_func(mapping_list, maj_data)

In [12]:
def choose_right_regions(data):
    dif_num_of_counties = []
    for region in data.region.unique():
        temp_data = data.loc[data.region == region]
        if len(temp_data.loc[temp_data.year == np.sort(temp_data.year.unique())[:(len(temp_data.year.unique()) - 2):-1][
            0], "county_num"].unique()) != len(temp_data.loc[temp_data.year == np.sort(temp_data.year.unique())[
                                                                               (len(temp_data.year.unique()) - 2)::-1][
            0], "county_num"].unique()):
            dif_num_of_counties.append(region)
    return dif_num_of_counties

In [13]:
regions = choose_right_regions(data)

In [14]:
regions

['Приморский край',
 'Мурманская область',
 'Новгородская область',
 'Костромская область',
 'Республика Алтай',
 'Республика Марий Эл',
 'Хабаровский край',
 'Тульская область']

In [15]:
dif_in_dif_data_2 = pd.DataFrame()

In [16]:
for region in regions:
    temp_data = maj_data.loc[maj_data.region == region]
    dif_in_dif_data_2 = dif_in_dif_data_2.append(temp_data)

In [17]:
dif_in_dif_data_2.head()

,cand_names,votes,uik,num_of_registered_voters,num_of_recieved_ballots,num_of_ballots_casted_anticipatorily,num_of_ballots_given_away_on_voting_day_indoors,num_of_ballots_given_away_on_voting_day_at_home,num_of_ballots_not_used,ballots_in_portable_ballot_box,ballots_in_stationary_ballot_box,num_of_not_valid_ballots,num_of_valid_ballots,num_of_lost_ballots,num_of_ballots_not_recorded_upon_receipt,region,birth_data,nom_subject,county_num,county_names,reg_status,electoral_status,year,name_of_elections,level,kind,type,elec_system,num_of_received_absentee_certificates,num_of_given_absentee_certificates,num_of_voted_by_absentee_certificates,num_of_absentee_certificates_not_used,num_of_given_by_TIK(OIK)_absentee_certificates,num_of_lost_absentee_certificates,num_of_absentee_certificates_not_recorded_upon_receipt
663481,Долгачев Анатолий Николаевич,226.0,УИК №201,2571,2500,0.0,938.0,12,1550,12,938,52,898,0,0,Приморский край,01.12.1986,КПРФ,1,№ 1,True,избр.,2011,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,25.0,9.0,1.0,16.0,2.0,0.0,NaN
663482,Долгачев Анатолий Николаевич,260.0,УИК №202,3009,2600,0.0,903.0,18,1679,18,885,39,864,0,0,Приморский край,01.12.1986,КПРФ,1,№ 1,True,избр.,2011,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,25.0,21.0,25.0,4.0,3.0,0.0,NaN
663483,Долгачев Анатолий Николаевич,241.0,УИК №203,1931,1900,0.0,857.0,8,1033,8,855,40,823,2,0,Приморский край,01.12.1986,КПРФ,1,№ 1,True,избр.,2011,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,25.0,22.0,6.0,3.0,1.0,0.0,NaN
663484,Долгачев Анатолий Николаевич,310.0,УИК №204,2358,2300,0.0,957.0,24,1319,24,957,47,934,0,0,Приморский край,01.12.1986,КПРФ,1,№ 1,True,избр.,2011,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,25.0,22.0,28.0,3.0,1.0,0.0,NaN
663485,Долгачев Анатолий Николаевич,222.0,УИК №205,1979,2000,0.0,793.0,17,1190,17,793,43,767,0,0,Приморский край,01.12.1986,КПРФ,1,№ 1,True,избр.,2011,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,25.0,9.0,4.0,16.0,0.0,0.0,NaN


In [18]:
def add_info(dif_in_dif):
    incum_dataset = pd.DataFrame()
    id_counter = 1
    for region in dif_in_dif.region.unique():
        temp_data = dif_in_dif.loc[dif_in_dif.region == region].copy()
        temp_data["treatment_period"] = [1 if year > 2018 else 0 for year in temp_data.year]
        temp_data["treatment"], temp_data["incumbent"], temp_data["first_time_incumbent"] = np.NaN, np.NaN, np.NaN
        uni_years = np.sort(temp_data.year.unique())[::-1]
        #print(uni_years)
        
        for name in temp_data.loc[(temp_data.year == uni_years[1]) & (~pd.isnull(temp_data.electoral_status)), "cand_names"].unique():
            old_subset = temp_data.loc[(temp_data.year == uni_years[1]) & (temp_data.cand_names == name), "uik"].unique()
            new_subset = temp_data.loc[(temp_data.year == uni_years[0]) & (temp_data.cand_names == name), "uik"].unique()
            if len(uni_years) > 2:
                very_old_subset = temp_data.loc[(temp_data.year == uni_years[2]) & (temp_data.cand_names == name), "uik"].unique()
            else:
                very_old_subset = []
            for uik in old_subset:
                temp_data.loc[temp_data.uik == uik, "incumbent"] = name
                if uik in very_old_subset:
                    temp_data.loc[temp_data.uik == uik, "first_time_incumbent"] = 0
                else:
                    temp_data.loc[temp_data.uik == uik, "first_time_incumbent"] = 1
                    
            temp_data.loc[series_in(temp_data.uik, new_subset), "district_id"] = id_counter
            id_counter += 1
                
            if len(new_subset) == 0:
                continue
            uiks_data = pd.DataFrame({"uik": pd.Series(new_subset), "treat": pd.Series([0 if i in old_subset else 1
                                                                                        for i in new_subset])})
            if sum(uiks_data.treat)/len(uiks_data) != 0 and sum(uiks_data.treat)/len(uiks_data) != 1:
                for uik in uiks_data.uik:
                    temp_data.loc[temp_data.uik == uik, "treatment"] = uiks_data.loc[uiks_data.uik == uik,
                                                                                     "treat"].iloc[0]
        temp_data.drop(index=temp_data.loc[pd.isnull(temp_data.treatment)].index, inplace=True)
        incum_dataset = incum_dataset.append(temp_data.loc[temp_data.year >= uni_years[1]])
    #incum_dataset.to_csv("D:/DZ/Course_5/Курсовая/data/dif_in_dif_incum/incum_dataset_v4.0.csv")
    return incum_dataset

In [19]:
incum_dataset = add_info(dif_in_dif_data_2)

In [20]:
incum_dataset.head()

,cand_names,votes,uik,num_of_registered_voters,num_of_recieved_ballots,num_of_ballots_casted_anticipatorily,num_of_ballots_given_away_on_voting_day_indoors,num_of_ballots_given_away_on_voting_day_at_home,num_of_ballots_not_used,ballots_in_portable_ballot_box,ballots_in_stationary_ballot_box,num_of_not_valid_ballots,num_of_valid_ballots,num_of_lost_ballots,num_of_ballots_not_recorded_upon_receipt,region,birth_data,nom_subject,county_num,county_names,reg_status,electoral_status,year,name_of_elections,level,kind,type,elec_system,num_of_received_absentee_certificates,num_of_given_absentee_certificates,num_of_voted_by_absentee_certificates,num_of_absentee_certificates_not_used,num_of_given_by_TIK(OIK)_absentee_certificates,num_of_lost_absentee_certificates,num_of_absentee_certificates_not_recorded_upon_receipt,treatment_period,treatment,incumbent,first_time_incumbent,district_id
871623,Башко Виталий Русланович,66.0,УИК №601,2101,1700,0.0,751.0,8,941,8,751,53,706,0,0,Приморский край,06.10.1987,Справедливая Россия,2,Одномандатный № 2,True,NaN,2021,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,Новиков Дмитрий Геннадьевич,1.0,2.0
871624,Башко Виталий Русланович,39.0,УИК №602,1847,1700,0.0,484.0,15,1201,15,481,24,472,0,0,Приморский край,06.10.1987,Справедливая Россия,2,Одномандатный № 2,True,NaN,2021,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,Новиков Дмитрий Геннадьевич,1.0,2.0
871625,Башко Виталий Русланович,61.0,УИК №603,2296,2000,0.0,727.0,14,1259,14,727,44,697,0,0,Приморский край,06.10.1987,Справедливая Россия,2,Одномандатный № 2,True,NaN,2021,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,Новиков Дмитрий Геннадьевич,1.0,2.0
871626,Башко Виталий Русланович,53.0,УИК №604,2000,1700,0.0,726.0,22,952,22,704,33,693,0,0,Приморский край,06.10.1987,Справедливая Россия,2,Одномандатный № 2,True,NaN,2021,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,Новиков Дмитрий Геннадьевич,1.0,2.0
871627,Башко Виталий Русланович,44.0,УИК №605,1728,1500,0.0,542.0,15,943,15,542,26,531,0,0,Приморский край,06.10.1987,Справедливая Россия,2,Одномандатный № 2,True,NaN,2021,Выборы депутатов Законодательного Собрания При...,Региональный,Выборы депутата,Основные,Смешанная - пропорциональная и мажоритарная,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0.0,Новиков Дмитрий Геннадьевич,1.0,2.0


In [21]:
incum_dataset.year.unique()

array([2021, 2016, 2020, 2015, 2019, 2014], dtype=int64)

In [22]:
def quick_fix(data): # correct one
    final_data = pd.DataFrame()
    incumbents = data.incumbent.unique()
    for i in pd.Series(zip(data.region, data.year, data.uik)).unique():
        if i[2] == "Дистанционное электронное голосование":
            continue
        temp_data = data.loc[(data.region == i[0]) & (data.year == i[1]) & (data.uik == i[2])]
        treatment_info = pd.Series(zip(temp_data.treatment, temp_data.treatment_period)).unique()
        if len(treatment_info) > 1:
            print("Breakpoint")
            breakpoint()
        if temp_data.num_of_registered_voters.sum() == 0:
            continue
        if len(temp_data.incumbent.unique()) > 1:
            print("Breakpoint")
            breakpoint()
        vts = temp_data.loc[temp_data.cand_names == temp_data.incumbent.unique()[0], "votes"]
        if len(vts) == 0:
            incum_vote = temp_data.loc[series_in(temp_data.cand_names, incumbents), "votes"]
            if len(incum_vote) > 1 or len(incum_vote) == 0:
                print("Houston, we got a problem", i)
                print(len(incum_vote))
                break
            #print(test.iloc[0])
            dif=(incum_vote.iloc[0] - 
                   temp_data.loc[~series_in(temp_data.cand_names, incumbents), "votes"].sort_values(ascending=False).iloc[0]) / temp_data.votes.sum()
        else:
            dif = (temp_data.loc[temp_data.cand_names == temp_data.incumbent.unique()[0], "votes"].iloc[0] - 
                       temp_data.loc[temp_data.cand_names != temp_data.incumbent.unique()[0], "votes"].sort_values(ascending=False).iloc[0]) / temp_data.votes.sum()
        final_data = final_data.append(pd.DataFrame({"region": pd.Series(i[0]), "year": pd.Series(i[1]),
                                                     "uik": pd.Series(i[2]), "dif": pd.Series(dif),
                                                     "treatment": treatment_info[0][0],
                                                     "treatment_period": treatment_info[0][1], 
                                                     "incumbent": temp_data.incumbent.unique(),
                                                     "district_id": temp_data.district_id.unique(),
                                                     "first_time_incumbent": temp_data.first_time_incumbent.unique()}))
    final_data.reset_index(drop=True, inplace=True)
    #final_data.to_csv("D:/DZ/Course_5/Курсовая/data/dif_in_dif_incum/incum_dataset_v5.0.csv")
    return final_data

In [23]:
final_data = quick_fix(incum_dataset)

In [24]:
final_data.head()

,region,year,uik,dif,treatment,treatment_period,incumbent,district_id,first_time_incumbent
0,Приморский край,2021,УИК №601,-0.026912,0.0,1,Новиков Дмитрий Геннадьевич,2.0,1.0
1,Приморский край,2021,УИК №602,0.002119,0.0,1,Новиков Дмитрий Геннадьевич,2.0,1.0
2,Приморский край,2021,УИК №603,0.133429,0.0,1,Новиков Дмитрий Геннадьевич,2.0,1.0
3,Приморский край,2021,УИК №604,-0.083694,0.0,1,Новиков Дмитрий Геннадьевич,2.0,1.0
4,Приморский край,2021,УИК №605,-0.018832,0.0,1,Новиков Дмитрий Геннадьевич,2.0,1.0


In [25]:
final_data["competitive"] = np.NaN
for region in final_data.region.unique():
    if region in ["Приморский край", "Мурманская область", "Костромская область", "Республика Алтай", "Хабаровский край", "Новгородская область"]:
        final_data.loc[final_data.region == region, "competitive"] = 1
    else:
        final_data.loc[final_data.region == region, "competitive"] = 0

In [26]:
final_data["disctrict_incumbent"] = np.NaN
for _id in final_data.district_id.unique():
    inc = final_data.loc[(final_data.district_id == _id) & (final_data.treatment == 0), "incumbent"].unique()
    #print(_id, inc)
    if len(inc) > 1:
        print("Problem found")
        break
    if sum(pd.isnull(final_data.loc[(final_data.district_id == _id), "incumbent"].unique())) > 0:
        #print("Yes")
        final_data.drop(index = final_data.loc[(final_data.district_id == _id) & pd.isnull(final_data.incumbent)].index, inplace=True)
        if sum(final_data.loc[final_data.district_id == _id, "treatment"]) == 0 or sum(final_data.loc[final_data.district_id == _id, "treatment"]) == len(final_data):
            #print("Yes again")
            final_data.drop(index = final_data.loc[final_data.district_id == _id].index, inplace=True)
    final_data.loc[final_data.district_id == _id, "disctrict_incumbent"] = inc[0]

In [27]:
final_data["united_russia_cand"] = np.NaN
for cand in final_data.disctrict_incumbent.unique():
    if incum_dataset.loc[incum_dataset.cand_names == cand, "nom_subject"].unique()[0] == "Единая Россия":
        final_data.loc[final_data.disctrict_incumbent == cand, "united_russia_cand"] = 1
    else:
        final_data.loc[final_data.disctrict_incumbent == cand, "united_russia_cand"] = 0

In [28]:
def verification(maj_data, final_data):
    final_data.verification, final_data.incum_participation_in_t = np.NaN, np.NaN
    for i in pd.Series(zip(final_data.region, final_data.uik)).unique():
        temp_data = maj_data.loc[(maj_data.region == i[0]) & (maj_data.uik == i[1])].sort_values(by="year")
        winners = temp_data.loc[~pd.isnull(temp_data.electoral_status), 'cand_names']
        if len(winners) < 3:
            if len(winners) < 2:
                print("<2 Error", i)
                break
            else:
                #print("less_than_three", i)
                final_data.loc[(final_data.region == i[0]) & (final_data.uik == i[1]), "verification"] = 1
                final_data.loc[(final_data.region == i[0]) & (final_data.uik == i[1]), "incum_participation_in_t"] = 0
                continue
        if winners.iloc[0] != winners.iloc[1]:
            final_data.loc[(final_data.region == i[0]) & (final_data.uik == i[1]), "verification"] = 1
        else:
            final_data.loc[(final_data.region == i[0]) & (final_data.uik == i[1]), "verification"] = 0
        if winners.iloc[0] in temp_data.loc[temp_data.year == temp_data.year.sort_values().unique()[1], "cand_names"].unique():
            final_data.loc[(final_data.region == i[0]) & (final_data.uik == i[1]), "incum_participation_in_t"] = 1
        else:
            final_data.loc[(final_data.region == i[0]) & (final_data.uik == i[1]), "incum_participation_in_t"] = 0
    return final_data

In [29]:
outp = verification(maj_data, final_data)

In [30]:
final_data.first_time_incumbent = outp["verification"]

In [31]:
non_scalable_ie = final_data.drop(index = final_data.loc[(final_data.first_time_incumbent == 1) & (final_data["incum_participation_in_t"] == 1)].index)
scalable_ie = final_data.drop(index = final_data.loc[final_data["incum_participation_in_t"] == 1].index)

In [32]:
def filter_data(data):
    output_fti, output_not_fti = pd.DataFrame(), pd.DataFrame()
    for i in data.district_id.unique():
        temp_data = data.loc[data.district_id == i].copy()
        fti, not_fti = temp_data.loc[temp_data.first_time_incumbent == 1], temp_data.loc[temp_data.first_time_incumbent == 0]
        #print("fti:", sum(fti.treatment), len(fti), i)
        #print("not_fti:", sum(not_fti.treatment), len(not_fti), i)
        if 0 < sum(fti.treatment) < len(fti):
            output_fti = output_fti.append(fti)
        if 0 < sum(not_fti.treatment) < len(not_fti):
            output_not_fti = output_not_fti.append(not_fti)
    return output_fti, output_not_fti

In [33]:
final_data.district_id.unique()

array([  2.,  12.,  13.,   5.,   7.,  11.,  16.,  18.,  21.,  29.,  22.,
        25.,  24.,  28.,  27.,  32.,  35.,  40.,  41.,  42.,  47.,  51.,
        52.,  53.,  58.,  61.,  64.,  70.,  73.,  76.,  78.,  92.,  97.,
       107., 124., 122., 123., 129., 143., 144., 137., 150., 149., 148.,
       135., 142., 152., 153.])

In [34]:
final_data_filtered, final_data_unfiltered = filter_data(non_scalable_ie)

In [38]:
final_data_filtered.loc[final_data_filtered.year > 2018].groupby(["district_id", "region"])["dif"].mean()

district_id  region              
2.0          Приморский край        -0.060520
5.0          Приморский край         0.347485
7.0          Приморский край         0.204814
11.0         Приморский край         0.131200
12.0         Приморский край         0.149762
13.0         Приморский край        -0.026095
16.0         Приморский край         0.059726
18.0         Приморский край         0.189113
21.0         Мурманская область      0.199240
22.0         Мурманская область      0.005230
24.0         Мурманская область      0.082878
25.0         Мурманская область      0.119280
27.0         Мурманская область      0.098052
28.0         Мурманская область      0.060701
29.0         Мурманская область     -0.107108
32.0         Мурманская область     -0.026960
35.0         Мурманская область      0.088740
40.0         Новгородская область    0.093813
41.0         Новгородская область    0.092822
42.0         Новгородская область   -0.003892
47.0         Новгородская область   -0.210842


In [35]:
model_fil_1 = sm_f.ols(formula="dif ~ treatment_period*treatment", data=final_data_filtered).fit(cov_type='HC3')

In [36]:
model_fil_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     110.2
Date:                Sat, 17 Sep 2022   Prob (F-statistic):           1.91e-67
Time:                        16:42:55   Log-Likelihood:                -814.08
No. Observations:                2818   AIC:                             1636.
Df Residuals:                    2814   BIC:                             1660.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.4316      0.010     43.479      0.000       0.412       0.451
treatment_period              -0.1962      0.015    -12.945      0.000      -0.226      -0.166
treatment                     -0.1452      0.016     -8.803      0.000      -0.178      -0.113
treatment_period:treatment     0.0380      0.025      1.494      0.135      -0.012       0.088
==============================================================================
Omnibus:                       44.543   Durbin-Watson:                   0.630
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.265
Skew:                          -0.309   Prob(JB):                     8.99e-11
Kurtosis:                       3.113   Cond. No.                         6.36
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [60]:
model_fil_2 = sm_f.ols(formula="dif ~ treatment_period*treatment + C(region)", data=final_data_filtered).fit(cov_type='HC3')

In [61]:
model_fil_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     186.6
Date:                Sat, 17 Sep 2022   Prob (F-statistic):          1.03e-301
Time:                        15:41:45   Log-Likelihood:                -277.66
No. Observations:                2818   AIC:                             577.3
Df Residuals:                    2807   BIC:                             642.7
Df Model:                          10                                         
Covariance Type:                  HC3                                         
=====================================================================================================
                                        coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.4519      0.021     21.834      0.000       0.411       0.493
C(region)[T.Мурманская область]      -0.1890      0.023     -8.163      0.000      -0.234      -0.144
C(region)[T.Новгородская область]    -0.0934      0.023     -3.993      0.000      -0.139      -0.048
C(region)[T.Приморский край]         -0.1162      0.021     -5.436      0.000      -0.158      -0.074
C(region)[T.Республика Алтай]        -0.2565      0.054     -4.714      0.000      -0.363      -0.150
C(region)[T.Республика Марий Эл]     -0.1564      0.064     -2.446      0.014      -0.282      -0.031
C(region)[T.Тульская область]         0.2234      0.020     11.051      0.000       0.184       0.263
C(region)[T.Хабаровский край]        -0.3847      0.038    -10.209      0.000      -0.459      -0.311
treatment_period                     -0.1962      0.012    -16.086      0.000      -0.220      -0.172
treatment                            -0.1260      0.014     -8.869      0.000      -0.154      -0.098
treatment_period:treatment            0.0208      0.022      0.968      0.333      -0.021       0.063
==============================================================================
Omnibus:                       66.430   Durbin-Watson:                   0.921
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              142.398
Skew:                          -0.086   Prob(JB):                     1.20e-31
Kurtosis:                       4.088   Cond. No.                         16.8
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [56]:
model_fil_3 = sm_f.ols(formula="dif ~ treatment_period*treatment + C(district_id)", data=final_data_filtered).fit(cov_type='HC3')

In [57]:
model_fil_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.493
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     103.4
Date:                Sat, 17 Sep 2022   Prob (F-statistic):               0.00
Time:                        15:41:34   Log-Likelihood:                -7.5706
No. Observations:                2818   AIC:                             113.1
Df Residuals:                    2769   BIC:                             404.4
Df Model:                          48                                         
Covariance Type:                  HC3                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.1297      0.020      6.496      0.000       0.091       0.169
C(district_id)[T.5.0]          0.3567      0.033     10.688      0.000       0.291       0.422
C(district_id)[T.7.0]          0.3056      0.032      9.677      0.000       0.244       0.368
C(district_id)[T.11.0]         0.0972      0.031      3.157      0.002       0.037       0.158
C(district_id)[T.12.0]         0.2228      0.037      6.080      0.000       0.151       0.295
C(district_id)[T.13.0]        -0.0030      0.034     -0.089      0.929      -0.069       0.063
C(district_id)[T.16.0]         0.3010      0.031      9.867      0.000       0.241       0.361
C(district_id)[T.18.0]         0.1762      0.029      5.989      0.000       0.119       0.234
C(district_id)[T.21.0]         0.2256      0.025      9.058      0.000       0.177       0.274
C(district_id)[T.22.0]         0.0718      0.023      3.140      0.002       0.027       0.117
C(district_id)[T.24.0]         0.1578      0.022      7.069      0.000       0.114       0.202
C(district_id)[T.25.0]         0.2054      0.047      4.383      0.000       0.114       0.297
C(district_id)[T.27.0]         0.1832      0.034      5.450      0.000       0.117       0.249
C(district_id)[T.28.0]         0.1124      0.030      3.759      0.000       0.054       0.171
C(district_id)[T.29.0]         0.0831      0.036      2.340      0.019       0.014       0.153
C(district_id)[T.32.0]         0.0257      0.035      0.725      0.469      -0.044       0.095
C(district_id)[T.35.0]         0.2422      0.034      7.153      0.000       0.176       0.309
C(district_id)[T.40.0]         0.1154      0.031      3.753      0.000       0.055       0.176
C(district_id)[T.41.0]         0.0893      0.031      2.852      0.004       0.028       0.151
C(district_id)[T.42.0]         0.0134      0.031      0.427      0.669      -0.048       0.075
C(district_id)[T.47.0]         0.0406      0.036      1.134      0.257      -0.030       0.111
C(district_id)[T.51.0]         0.2364      0.026      9.149      0.000       0.186       0.287
C(district_id)[T.52.0]         0.4442      0.026     17.354      0.000       0.394       0.494
C(district_id)[T.53.0]         0.3016      0.032      9.362      0.000       0.238       0.365
C(district_id)[T.58.0]         0.1443      0.062      2.315      0.021       0.022       0.266
C(district_id)[T.61.0]         0.3459      0.037      9.355      0.000       0.273       0.418
C(district_id)[T.64.0]         0.2524      0.044      5.731      0.000       0.166       0.339
C(district_id)[T.70.0]         0.3657      0.038      9.627      0.000       0.291       0.440
C(district_id)[T.78.0]         0.0584      0.054      1.072      0.284      -0.048       0.165
C(district_id)[T.92.0]         0.0970      

In [64]:
model_fil_competitive = sm_f.ols(formula="dif ~ treatment_period*treatment + C(district_id)", data=final_data_filtered.loc[final_data_filtered.competitive == 1]).fit(cov_type='HC3')

In [65]:
model_fil_competitive.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.299
Model:                            OLS   Adj. R-squared:                  0.286
Method:                 Least Squares   F-statistic:                     33.11
Date:                Sat, 17 Sep 2022   Prob (F-statistic):          1.79e-168
Time:                        15:43:34   Log-Likelihood:                -158.98
No. Observations:                1916   AIC:                             390.0
Df Residuals:                    1880   BIC:                             590.0
Df Model:                          35                                         
Covariance Type:                  HC3                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.1225      0.021      5.921      0.000       0.082       0.163
C(district_id)[T.5.0]          0.3548      0.033     10.667      0.000       0.290       0.420
C(district_id)[T.7.0]          0.3037      0.032      9.564      0.000       0.241       0.366
C(district_id)[T.11.0]         0.1044      0.030      3.475      0.001       0.046       0.163
C(district_id)[T.12.0]         0.2264      0.036      6.316      0.000       0.156       0.297
C(district_id)[T.13.0]        -0.0074      0.034     -0.220      0.826      -0.073       0.058
C(district_id)[T.16.0]         0.3102      0.031      9.954      0.000       0.249       0.371
C(district_id)[T.18.0]         0.1786      0.029      6.177      0.000       0.122       0.235
C(district_id)[T.21.0]         0.2242      0.025      9.048      0.000       0.176       0.273
C(district_id)[T.22.0]         0.0670      0.023      2.858      0.004       0.021       0.113
C(district_id)[T.24.0]         0.1641      0.022      7.341      0.000       0.120       0.208
C(district_id)[T.25.0]         0.2122      0.047      4.485      0.000       0.119       0.305
C(district_id)[T.27.0]         0.1896      0.034      5.591      0.000       0.123       0.256
C(district_id)[T.28.0]         0.1155      0.030      3.813      0.000       0.056       0.175
C(district_id)[T.29.0]         0.0981      0.036      2.751      0.006       0.028       0.168
C(district_id)[T.32.0]         0.0291      0.034      0.855      0.392      -0.038       0.096
C(district_id)[T.35.0]         0.2433      0.034      7.179      0.000       0.177       0.310
C(district_id)[T.40.0]         0.1189      0.030      3.975      0.000       0.060       0.178
C(district_id)[T.41.0]         0.0897      0.030      2.977      0.003       0.031       0.149
C(district_id)[T.42.0]         0.0107      0.031      0.341      0.733      -0.051       0.072
C(district_id)[T.47.0]         0.0391      0.037      1.050      0.294      -0.034       0.112
C(district_id)[T.51.0]         0.2387      0.025      9.467      0.000       0.189       0.288
C(district_id)[T.52.0]         0.4497      0.025     17.834      0.000       0.400       0.499
C(district_id)[T.53.0]         0.3047      0.030     10.047      0.000       0.245       0.364
C(district_id)[T.58.0]         0.1413      0.061      2.330      0.020       0.022       0.260
C(district_id)[T.61.0]         0.3581      0.036      9.851      0.000       0.287       0.429
C(district_id)[T.64.0]         0.2673      0.043      6.162      0.000       0.182       0.352
C(district_id)[T.70.0]         0.3625      0.037      9.679      0.000       0.289       0.436
C(district_id)[T.78.0]         0.0693      0.055      1.271      0.204      -0.038       0.176
C(district_id)[T.122.0]       -0.0180      

In [66]:
model_fil_noncompetitive = sm_f.ols(formula="dif ~ treatment_period*treatment + C(district_id)", data=final_data_filtered.loc[final_data_filtered.competitive == 0]).fit(cov_type='HC3')

In [67]:
model_fil_noncompetitive.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     65.04
Date:                Sat, 17 Sep 2022   Prob (F-statistic):          1.94e-131
Time:                        15:46:02   Log-Likelihood:                 228.63
No. Observations:                 902   AIC:                            -425.3
Df Residuals:                     886   BIC:                            -348.4
Df Model:                          15                                         
Covariance Type:                  HC3                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.2237      0.419      0.534      0.594      -0.598       1.046
C(district_id)[T.97.0]         0.0257      0.428      0.060      0.952      -0.813       0.864
C(district_id)[T.107.0]        0.1048      0.424      0.247      0.805      -0.725       0.935
C(district_id)[T.135.0]        0.4487      0.423      1.061      0.289      -0.380       1.278
C(district_id)[T.137.0]        0.4835      0.421      1.150      0.250      -0.341       1.308
C(district_id)[T.142.0]        0.3292      0.422      0.780      0.435      -0.498       1.156
C(district_id)[T.143.0]        0.4376      0.421      1.039      0.299      -0.388       1.263
C(district_id)[T.144.0]        0.4973      0.421      1.182      0.237      -0.327       1.322
C(district_id)[T.148.0]        0.5782      0.420      1.376      0.169      -0.245       1.402
C(district_id)[T.149.0]        0.5499      0.423      1.300      0.193      -0.279       1.379
C(district_id)[T.150.0]        0.2768      0.421      0.657      0.511      -0.549       1.102
C(district_id)[T.152.0]        0.1148      0.426      0.270      0.788      -0.720       0.949
C(district_id)[T.153.0]        0.4367      0.425      1.028      0.304      -0.396       1.269
treatment_period              -0.2505      0.015    -17.252      0.000      -0.279      -0.222
treatment                     -0.0569      0.021     -2.698      0.007      -0.098      -0.016
treatment_period:treatment     0.0235      0.028      0.824      0.410      -0.032       0.079
==============================================================================
Omnibus:                       61.764   Durbin-Watson:                   1.182
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              115.844
Skew:                          -0.461   Prob(JB):                     6.99e-26
Kurtosis:                       4.494   Cond. No.                         70.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

In [68]:
model_fil_ur = sm_f.ols(formula="dif ~ treatment_period*treatment + C(district_id)", data=final_data_filtered.loc[final_data_filtered.united_russia_cand == 1]).fit(cov_type='HC3')

In [69]:
model_fil_ur.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.494
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     105.9
Date:                Sat, 17 Sep 2022   Prob (F-statistic):               0.00
Time:                        15:48:58   Log-Likelihood:                -19.370
No. Observations:                2776   AIC:                             132.7
Df Residuals:                    2729   BIC:                             411.4
Df Model:                          46                                         
Covariance Type:                  HC3                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.1319      0.020      6.579      0.000       0.093       0.171
C(district_id)[T.5.0]          0.3566      0.033     10.664      0.000       0.291       0.422
C(district_id)[T.7.0]          0.3056      0.032      9.670      0.000       0.244       0.368
C(district_id)[T.11.0]         0.0974      0.031      3.155      0.002       0.037       0.158
C(district_id)[T.12.0]         0.2229      0.037      6.073      0.000       0.151       0.295
C(district_id)[T.13.0]        -0.0031      0.034     -0.092      0.927      -0.069       0.063
C(district_id)[T.16.0]         0.3013      0.031      9.870      0.000       0.241       0.361
C(district_id)[T.18.0]         0.1763      0.030      5.976      0.000       0.118       0.234
C(district_id)[T.21.0]         0.2256      0.025      8.997      0.000       0.176       0.275
C(district_id)[T.22.0]         0.0716      0.023      3.123      0.002       0.027       0.117
C(district_id)[T.24.0]         0.1579      0.022      7.026      0.000       0.114       0.202
C(district_id)[T.25.0]         0.2055      0.047      4.378      0.000       0.114       0.298
C(district_id)[T.27.0]         0.1834      0.034      5.437      0.000       0.117       0.250
C(district_id)[T.28.0]         0.1124      0.030      3.737      0.000       0.053       0.171
C(district_id)[T.29.0]         0.0831      0.036      2.338      0.019       0.013       0.153
C(district_id)[T.32.0]         0.0258      0.036      0.725      0.469      -0.044       0.095
C(district_id)[T.35.0]         0.2422      0.034      7.171      0.000       0.176       0.308
C(district_id)[T.40.0]         0.1154      0.031      3.726      0.000       0.055       0.176
C(district_id)[T.42.0]         0.0134      0.032      0.422      0.673      -0.049       0.076
C(district_id)[T.47.0]         0.0406      0.036      1.138      0.255      -0.029       0.110
C(district_id)[T.51.0]         0.2365      0.026      9.115      0.000       0.186       0.287
C(district_id)[T.52.0]         0.4443      0.026     17.310      0.000       0.394       0.495
C(district_id)[T.53.0]         0.3017      0.032      9.290      0.000       0.238       0.365
C(district_id)[T.61.0]         0.3461      0.037      9.345      0.000       0.274       0.419
C(district_id)[T.64.0]         0.2528      0.044      5.724      0.000       0.166       0.339
C(district_id)[T.70.0]         0.3657      0.038      9.600      0.000       0.291       0.440
C(district_id)[T.78.0]         0.0586      0.055      1.075      0.283      -0.048       0.166
C(district_id)[T.92.0]         0.0971      0.430      0.226      0.821      -0.746       0.940
C(district_id)[T.97.0]         0.1228      0.070      1.742      0.081      -0.015       0.261
C(district_id)[T.107.0]        0.1826      

In [70]:
model_fil_nonur = sm_f.ols(formula="dif ~ treatment_period*treatment + C(district_id)", data=final_data_filtered.loc[final_data_filtered.united_russia_cand == 0]).fit(cov_type='HC3')

In [71]:
model_fil_nonur.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.121
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     1.291
Date:                Sat, 17 Sep 2022   Prob (F-statistic):              0.291
Time:                        15:50:38   Log-Likelihood:                 29.544
No. Observations:                  42   AIC:                            -49.09
Df Residuals:                      37   BIC:                            -40.40
Df Model:                           4                                         
Covariance Type:                  HC3                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.1037      0.034      3.095      0.002       0.038       0.169
C(district_id)[T.58.0]         0.0602      0.056      1.074      0.283      -0.050       0.170
treatment_period               0.0136      0.050      0.271      0.787      -0.085       0.112
treatment                     -0.0684      0.068     -0.999      0.318      -0.202       0.066
treatment_period:treatment    -0.0011      0.090     -0.012      0.990      -0.177       0.175
==============================================================================
Omnibus:                       14.671   Durbin-Watson:                   1.836
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               25.547
Skew:                          -0.877   Prob(JB):                     2.83e-06
Kurtosis:                       6.395   Cond. No.                         6.85
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""

# Scalable effect

In [38]:
def filter_data_scalable(data): # for general incumbency effect
    output_treat, output_not_treat = pd.DataFrame(), pd.DataFrame()
    for i in data.district_id.unique():
        temp_data = data.loc[data.district_id == i].copy()
        treat, not_treat = temp_data.loc[temp_data.treatment == 1], temp_data.loc[temp_data.treatment == 0]
        if treat.first_time_incumbent.sum() > 0 and not_treat.first_time_incumbent.sum() < len(not_treat):
            output_treat = output_treat.append(treat.loc[treat.first_time_incumbent == 1].append(not_treat.loc[not_treat.first_time_incumbent == 0]))
        if treat.first_time_incumbent.sum() < len(treat) and not_treat.first_time_incumbent.sum() > 0:
            output_not_treat = output_not_treat.append(treat.loc[treat.first_time_incumbent == 0].append(not_treat.loc[not_treat.first_time_incumbent == 1]))
    return output_treat, output_not_treat

In [39]:
scalable_filtered, scalable_unfiltered = filter_data_scalable(final_data)

In [42]:
model_s_fil = sm_f.ols(formula="dif ~ treatment_period*treatment + C(district_id)", data=scalable_filtered).fit(cov_type='HC3')

In [43]:
model_s_fil.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    dif   R-squared:                       0.378
Model:                            OLS   Adj. R-squared:                  0.351
Method:                 Least Squares   F-statistic:                     15.35
Date:                Sat, 17 Sep 2022   Prob (F-statistic):           3.66e-17
Time:                        17:03:12   Log-Likelihood:                 2.1866
No. Observations:                 192   AIC:                             13.63
Df Residuals:                     183   BIC:                             42.94
Df Model:                           8                                         
Covariance Type:                  HC3                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.5595      0.076      7.354      0.000       0.410       0.709
C(district_id)[T.58.0]        -0.1463      0.109     -1.341      0.180      -0.360       0.067
C(district_id)[T.64.0]         0.0435      0.045      0.964      0.335      -0.045       0.132
C(district_id)[T.92.0]        -0.2024      0.176     -1.147      0.251      -0.548       0.143
C(district_id)[T.97.0]        -0.1526      0.069     -2.218      0.027      -0.287      -0.018
C(district_id)[T.124.0]       -0.3557      0.083     -4.260      0.000      -0.519      -0.192
treatment_period              -0.3683      0.046     -8.032      0.000      -0.458      -0.278
treatment                     -0.4107      0.082     -5.031      0.000      -0.571      -0.251
treatment_period:treatment     0.3156      0.074      4.294      0.000       0.172       0.460
==============================================================================
Omnibus:                       14.527   Durbin-Watson:                   1.382
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               40.810
Skew:                           0.129   Prob(JB):                     1.37e-09
Kurtosis:                       5.244   Cond. No.                         14.0
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC3)
"""